In [106]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("traffic").getOrCreate()
from pyspark.sql.functions import * 

In [121]:
df = spark.read.csv("Traffic_PickUp.csv",header=True, inferSchema=True)

In [108]:
#df.printSchema()

In [122]:
revised_df = df.select(["tpep_picku","id","pickup_lon","pickup_lat"])

In [123]:
pickup = df["tpep_picku"]
revised_df = revised_df.select(year(pickup).alias("year"),month(pickup).alias("month"),dayofmonth(pickup).alias('day'),hour(pickup).alias('hours'),"id","pickup_lon")
#revised_df.select(countDistinct("id")).show()
#revised_df.show()

In [128]:
id_column = revised_df.select(df["id"]).distinct()

In [112]:
#revised_df.show()

+----+-----+---+-----+---+-------------+
|year|month|day|hours| id|   pickup_lon|
+----+-----+---+-----+---+-------------+
|2015|   12| 17|   10|254|-73.988838196|
|2015|   12| 17|   10|309|-73.954147339|
|2015|   12| 17|   10|381|-73.974319458|
|2015|   12| 17|   10|309|-73.952445984|
|2015|   12| 17|   10|454|-73.988494873|
|2015|   12| 17|   10|454|-73.986076355|
|2015|   12| 17|   10|430|-73.978416443|
|2015|   12| 17|   10|258|-73.956924438|
|2015|   12| 17|   10|329| -73.98551178|
|2015|   12| 17|   10|402| -74.00579071|
|2015|   12| 17|   10|551|-74.007522583|
|2015|   12| 17|   10|378|-73.994003296|
|2015|   12| 17|   10|329|-73.985588074|
|2015|   12| 17|   10|380|-73.977714539|
|2015|   12| 17|   10|260|-73.948204041|
|2015|   12| 17|   10|308|-73.963226318|
|2015|   12| 17|   10|160|-73.947006226|
|2015|   12| 17|   10|284|-73.950744629|
|2015|   12| 17|   10|205| -73.97833252|
|2015|   12| 17|   10|309| -73.95426178|
+----+-----+---+-----+---+-------------+
only showing top

In [129]:
maximum_month = revised_df.agg({"month":"max"}).collect()[0][0]
minimum_month = revised_df.agg({"month":"min"}).collect()[0][0]

In [114]:
maximum_year = revised_df.agg({"year":"max"}).collect()[0][0]
minimum_year = revised_df.agg({"year":"min"}).collect()[0][0]

In [115]:
maximum_day = revised_df.agg({"day":"max"}).collect()[0][0]
minimum_day = revised_df.agg({"day":"min"}).collect()[0][0]

In [116]:
maximum_hour = revised_df.agg({"hours":"max"}).collect()[0][0]
minimum_hour = revised_df.agg({"hours":"min"}).collect()[0][0]

In [119]:
for y in range(minimum_year,maximum_year+1): #loop thorough years
    for m in range(minimum_month,maximum_month+1): #loop through months
        for d in range(minimum_day,maximum_day+1): #loop through days
            for h in range(minimum_hour,maximum_hour+1): #loop through hours
                Time = revised_df.filter(revised_df["hours"] == h)
                hours = Time.groupBy("id").count().withColumnRenamed("count","%i-%i-%i %i:00"%(y,m,d,h))
                id_column = id_column.join(hours,['id'],"full_outer")

id_column = id_column.orderBy("id")
#id_column.toPandas().to_csv('mycsv.csv')    

In [120]:
id_column.show()

+---+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+
| id|2015-12-17 10:00|2015-12-17 11:00|2015-12-17 12:00|2015-12-17 13:00|2015-12-17 14:00|2015-12-17 15:00|2015-12-17 10:00|2015-12-17 11:00|2015-12-17 12:00|2015-12-17 13:00|2015-12-17 14:00|2015-12-17 15:00|
+---+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+
|  0|             124|             552|             636|             707|             899|             625|             124|             552|             636|             707|             899|             625|
| 10|            null|               9|               3|               4|               3|               7|            null|               9|               3|  